# Chapter 14: Exception Handling – Writing Robust Code

No matter how carefully you write your code, things can go wrong at runtime. A file might be missing, a network connection could drop, or the user might enter invalid data. If you don't handle these situations, your program will crash, leaving users frustrated and potentially losing data.

**Exception handling** is the mechanism C# provides to deal with unexpected errors gracefully. Instead of letting the program terminate abruptly, you can catch exceptions, take corrective action (like retrying, logging, or notifying the user), and continue execution or shut down cleanly.

In this chapter, you'll learn:

- What exceptions are and how they work in .NET.
- The `try`, `catch`, and `finally` blocks.
- Common exception types in the .NET framework.
- How to **throw** exceptions to signal errors.
- Creating **custom exception classes** to represent domain‑specific errors.
- Using **exception filters** with the `when` keyword.
- Best practices for rethrowing exceptions while preserving stack traces.
- When to catch exceptions and when to let them propagate.
- A practical example that handles file I/O errors robustly.

By the end, you'll be able to write code that anticipates failures and handles them gracefully, making your applications more reliable and user‑friendly.

---

## 14.1 What Are Exceptions?

An **exception** is an object that represents an error or unexpected condition. When something goes wrong, your code (or the .NET runtime) can **throw** an exception. Unless the exception is caught, the program will terminate with an error message.

### The Exception Hierarchy

All exceptions in .NET derive from the `System.Exception` class. Some important derived classes:

- `SystemException` – base for exceptions thrown by the runtime (e.g., `NullReferenceException`, `IndexOutOfRangeException`).
- `ApplicationException` – historically used for application‑defined exceptions, but now it's recommended to derive directly from `Exception`.
- `IOException` – for I/O errors.
- `ArgumentException` – for invalid arguments, with further derived types like `ArgumentNullException` and `ArgumentOutOfRangeException`.
- `InvalidOperationException` – when an operation is not valid for the current state.

Knowing the hierarchy helps you catch specific exceptions and handle them appropriately.

---

## 14.2 The `try`‑`catch`‑`finally` Blocks

The primary mechanism for handling exceptions is the `try` block, followed by one or more `catch` blocks, and optionally a `finally` block.

### Basic `try`‑`catch`

```csharp
try
{
    // Code that might throw an exception
    int[] numbers = new int[5];
    numbers[10] = 42; // This will throw IndexOutOfRangeException
}
catch (IndexOutOfRangeException ex)
{
    Console.WriteLine("An index was out of range!");
    Console.WriteLine($"Error details: {ex.Message}");
}
```

When an exception occurs inside the `try` block, the runtime looks for a matching `catch` block. If found, that block executes. If not, the exception propagates up the call stack.

### Multiple `catch` Blocks

You can have multiple `catch` blocks for different exception types. They are evaluated in order, so place more specific exceptions first.

```csharp
try
{
    string? s = null;
    Console.WriteLine(s.Length); // Throws NullReferenceException
}
catch (NullReferenceException ex)
{
    Console.WriteLine("Null reference!");
}
catch (Exception ex)
{
    Console.WriteLine($"Some other error: {ex.Message}");
}
```

Always catch the most specific exception first; otherwise, a more general catch like `Exception` will match first, and the specific ones will never be reached.

### The `finally` Block

The `finally` block executes **regardless** of whether an exception was thrown or caught. It's the perfect place for cleanup code, such as closing files, releasing resources, or resetting state.

```csharp
FileStream? file = null;
try
{
    file = File.OpenRead("somefile.txt");
    // Work with the file...
}
catch (FileNotFoundException ex)
{
    Console.WriteLine("File not found.");
}
finally
{
    file?.Close(); // Always close the file if it was opened
}
```

Even if an exception is thrown and not caught, the `finally` block runs before the program terminates (unless the process is aborted abnormally). This ensures resources are released.

---

## 14.3 Common Exception Types

Knowing the common exceptions helps you write appropriate catch blocks.

| Exception | When It Occurs |
|-----------|----------------|
| `NullReferenceException` | You try to access a member (method, property, field) of a variable that is `null`. |
| `IndexOutOfRangeException` | You try to access an array or collection element with an index outside the valid range. |
| `ArgumentException` | A method is called with an invalid argument. |
| `ArgumentNullException` | A method argument is `null` when it shouldn't be. |
| `ArgumentOutOfRangeException` | An argument value is outside the allowable range (e.g., a negative number where positive is required). |
| `InvalidOperationException` | An operation is not valid given the object's current state (e.g., moving an iterator after it has finished). |
| `IOException` | An I/O error occurs (file not found, disk full, etc.). |
| `DivideByZeroException` | You attempt to divide an integer by zero. |
| `OverflowException` | An arithmetic operation overflows (in a `checked` context). |
| `FormatException` | A string cannot be parsed into the expected type (e.g., `int.Parse("abc")`). |
| `NotImplementedException` | A method or property has not been implemented (often used as a placeholder). |
| `TimeoutException` | A time‑limited operation times out. |

---

## 14.4 Throwing Exceptions

You can throw exceptions from your own code using the `throw` keyword. This is useful to signal that something unexpected has occurred or that a method was called with invalid arguments.

### Basic Throw

```csharp
public void Deposit(decimal amount)
{
    if (amount <= 0)
    {
        throw new ArgumentException("Amount must be positive", nameof(amount));
    }
    _balance += amount;
}
```

### Rethrowing Exceptions

Sometimes you catch an exception to log it, but then you want the exception to continue propagating. Be careful how you rethrow.

```csharp
try
{
    // some operation
}
catch (Exception ex)
{
    LogError(ex);
    throw; // Good – preserves stack trace
    // throw ex; // Bad – resets stack trace, losing original call site
}
```

Using `throw;` rethrows the original exception with its stack trace intact. Using `throw ex;` throws the same exception but resets the stack trace to the point of the `throw`, making debugging harder.

### Throw Expressions (C# 7)

Starting with C# 7, you can use `throw` as an expression in certain contexts, such as in expression‑bodied members or the null‑coalescing operator.

```csharp
public string Name
{
    get => _name;
    set => _name = value ?? throw new ArgumentNullException(nameof(value));
}
```

Or in a conditional expression:

```csharp
int value = input != null ? int.Parse(input) : throw new ArgumentNullException(nameof(input));
```

---

## 14.5 Creating Custom Exceptions

When you need to represent an error specific to your application, you can create a custom exception class. It should derive from `Exception` (or a suitable derived class) and follow the standard patterns:

- Provide a parameterless constructor.
- Provide a constructor that takes a message.
- Provide a constructor that takes a message and an inner exception.
- If your exception has extra data, provide properties and a constructor to set them.
- Mark it `[Serializable]` if you need serialization (e.g., for remoting or logging across app domains).

```csharp
[Serializable]
public class InsufficientFundsException : Exception
{
    public decimal CurrentBalance { get; }
    public decimal WithdrawalAmount { get; }

    public InsufficientFundsException() { }

    public InsufficientFundsException(string message) : base(message) { }

    public InsufficientFundsException(string message, Exception inner) 
        : base(message, inner) { }

    public InsufficientFundsException(decimal balance, decimal amount)
        : base($"Insufficient funds: balance {balance:C}, tried to withdraw {amount:C}")
    {
        CurrentBalance = balance;
        WithdrawalAmount = amount;
    }

    // For serialization
    protected InsufficientFundsException(
        System.Runtime.Serialization.SerializationInfo info,
        System.Runtime.Serialization.StreamingContext context) : base(info, context)
    {
        CurrentBalance = info.GetDecimal("CurrentBalance");
        WithdrawalAmount = info.GetDecimal("WithdrawalAmount");
    }

    public override void GetObjectData(
        System.Runtime.Serialization.SerializationInfo info,
        System.Runtime.Serialization.StreamingContext context)
    {
        base.GetObjectData(info, context);
        info.AddValue("CurrentBalance", CurrentBalance);
        info.AddValue("WithdrawalAmount", WithdrawalAmount);
    }
}
```

Then you can throw it:

```csharp
public void Withdraw(decimal amount)
{
    if (amount > _balance)
        throw new InsufficientFundsException(_balance, amount);
    // ...
}
```

Custom exceptions make your error handling more expressive and allow catching specific application‑level errors.

---

## 14.6 Exception Filters (`when`)

C# 6 introduced exception filters, allowing you to specify a condition that must be true for a `catch` block to execute. This is done with the `when` keyword.

```csharp
try
{
    // Some operation
}
catch (HttpRequestException ex) when (ex.StatusCode == HttpStatusCode.NotFound)
{
    Console.WriteLine("Resource not found (404).");
}
catch (HttpRequestException ex) when (ex.StatusCode == HttpStatusCode.Unauthorized)
{
    Console.WriteLine("Access denied (401).");
}
```

Exception filters are evaluated before the catch block runs, and if the condition is false, the catch block is skipped and the exception continues to be searched for a matching handler. This is useful for handling exceptions differently based on runtime data.

Another common use is logging with filters:

```csharp
catch (Exception ex) when (Log(ex)) // Log returns false, so we don't catch
{
    // This block never executes because Log returns false
}
```

Here `Log` is a method that logs the exception and returns `false`, so the exception isn't caught but logging still happens.

---

## 14.7 Best Practices for Exception Handling

### 1. Catch Only What You Can Handle
Don't catch exceptions you can't meaningfully handle. Let them propagate to a higher level that knows what to do. Catching `Exception` at a low level and ignoring it is dangerous – you might hide critical errors.

### 2. Use `finally` for Cleanup
Always release resources (files, network connections, database connections) in a `finally` block or use the `using` statement (which compiles to a `try`/`finally`).

### 3. Prefer Specific Exception Types
Catch `FileNotFoundException` rather than `Exception`. This makes your intent clear and avoids swallowing unrelated errors.

### 4. Don't Swallow Exceptions Silently
Empty catch blocks like `catch { }` hide errors. At minimum, log the exception.

### 5. Throw Early, Catch Late
Validate input at the beginning of a method and throw appropriate exceptions. Let exceptions bubble up to the top level (e.g., UI or main loop) where they can be logged and presented to the user.

### 6. Use the `using` Statement for IDisposable
Instead of explicit `try`/`finally`, use `using` which ensures `Dispose` is called even if an exception occurs.

```csharp
using (var file = File.OpenRead("file.txt"))
{
    // work with file
} // automatically closed
```

### 7. Preserve Stack Trace When Rethrowing
Always use `throw;` without an exception object to rethrow the current exception.

### 8. Avoid Exceptions for Control Flow
Exceptions are for exceptional conditions. Don't use them to manage regular program flow (like exiting a loop). They are expensive and make code harder to read.

### 9. Document Exceptions Thrown
In public methods, use XML comments with `<exception>` tags to document which exceptions callers should expect.

```csharp
/// <exception cref="ArgumentException">Thrown when the input is invalid.</exception>
public void Process(string input) { ... }
```

### 10. Consider Performance
Throwing and catching exceptions is relatively slow. If you have a frequently occurring condition (like a failed parse), consider using the `TryParse` pattern instead of throwing.

---

## 14.8 Putting It All Together: A File Processor with Robust Error Handling

Let's build a program that reads a file, processes each line (converting to integers and summing them), and handles various errors gracefully. We'll demonstrate custom exceptions, exception filters, and proper cleanup.

```csharp
using System;
using System.IO;
using System.Collections.Generic;

namespace ExceptionHandlingDemo
{
    // Custom exception for processing errors
    public class ProcessingException : Exception
    {
        public string FilePath { get; }
        public int LineNumber { get; }

        public ProcessingException(string message, string filePath, int lineNumber, Exception inner)
            : base(message, inner)
        {
            FilePath = filePath;
            LineNumber = lineNumber;
        }
    }

    class Program
    {
        static void Main()
        {
            string filePath = "numbers.txt";

            try
            {
                int sum = ProcessFile(filePath);
                Console.WriteLine($"Sum of numbers: {sum}");
            }
            catch (FileNotFoundException ex)
            {
                Console.WriteLine($"Error: The file '{ex.FileName}' was not found.");
            }
            catch (UnauthorizedAccessException ex)
            {
                Console.WriteLine($"Error: No permission to read the file. {ex.Message}");
            }
            catch (ProcessingException ex)
            {
                Console.WriteLine($"Processing error at line {ex.LineNumber} in {ex.FilePath}: {ex.Message}");
                if (ex.InnerException != null)
                {
                    Console.WriteLine($"Inner error: {ex.InnerException.Message}");
                }
            }
            catch (Exception ex)
            {
                Console.WriteLine($"An unexpected error occurred: {ex.Message}");
                // Log the exception (omitted for brevity)
            }
            finally
            {
                Console.WriteLine("File processing attempt completed.");
            }
        }

        static int ProcessFile(string path)
        {
            // Using statement ensures the file is closed even if exceptions occur
            using (var reader = new StreamReader(path))
            {
                int sum = 0;
                int lineNumber = 0;
                string? line;

                while ((line = reader.ReadLine()) != null)
                {
                    lineNumber++;
                    try
                    {
                        int number = ParseLine(line, lineNumber);
                        sum += number;
                    }
                    catch (FormatException ex)
                    {
                        // Wrap and rethrow as a ProcessingException with context
                        throw new ProcessingException(
                            $"Invalid number format at line {lineNumber}",
                            path,
                            lineNumber,
                            ex);
                    }
                    catch (OverflowException ex)
                    {
                        throw new ProcessingException(
                            $"Number too large at line {lineNumber}",
                            path,
                            lineNumber,
                            ex);
                    }
                }
                return sum;
            }
        }

        static int ParseLine(string line, int lineNumber)
        {
            // Trim and check for empty lines (maybe treat as 0 or skip)
            if (string.IsNullOrWhiteSpace(line))
            {
                // We could return 0, but let's demonstrate exception filter later
                throw new FormatException("Line is empty.");
            }

            // Use TryParse to avoid exception for expected errors, but we'll keep throwing for demo
            if (!int.TryParse(line, out int result))
            {
                throw new FormatException($"'{line}' is not a valid integer.");
            }
            return result;
        }
    }
}
```

**Explanation:**

- `ProcessingException` is a custom exception that captures the file path, line number, and inner exception, providing rich context.
- In `Main`, we have a cascade of `catch` blocks from specific to general. The `finally` always runs.
- The `using` statement ensures `StreamReader` is disposed even if an exception occurs.
- `ProcessFile` reads line by line. For each line, it calls `ParseLine` inside a nested `try`‑`catch`. If parsing fails, we catch the `FormatException` or `OverflowException` and wrap it in a `ProcessingException` with additional context, then rethrow.
- `ParseLine` uses `int.TryParse` to avoid throwing, but for demonstration, we still throw `FormatException` on invalid input. In real code, you might return a `bool` or use `TryParse` to avoid exception overhead.

This example shows how exceptions can provide detailed error information while still allowing the program to handle failures at appropriate levels.

---

## 14.9 Common Pitfalls

### 1. Catching `Exception` and Doing Nothing
```csharp
catch (Exception) { } // Bad – swallows all errors
```
Always at least log the exception.

### 2. Using Exceptions for Control Flow
```csharp
try { int.Parse(input); } catch { useDefault = true; } // Avoid
```
Use `TryParse` instead.

### 3. Not Cleaning Up Resources
If you open a file in a `try` block and an exception occurs before the close, the file may remain locked unless you use `finally` or `using`.

### 4. Throwing `new Exception()` Without Context
Always use a specific exception type and include a meaningful message.

### 5. Losing Stack Trace
```csharp
catch (Exception ex) { throw ex; } // Bad – stack trace reset
```
Use `throw;` without the exception variable.

### 6. Ignoring Inner Exceptions
When wrapping an exception, always pass the original as the inner exception. This preserves the root cause.

### 7. Catching Exceptions You Can't Handle
Catching `OutOfMemoryException` is rarely useful – there's little you can do. Let it terminate the process.

---

## 14.10 Chapter Summary

In this chapter, you learned how to make your C# programs robust in the face of errors:

- **Exceptions** are objects that represent errors, with a rich hierarchy.
- **`try`‑`catch`** blocks allow you to handle exceptions. Multiple `catch` blocks let you handle different types.
- **`finally`** ensures cleanup code always runs.
- You can **throw** exceptions to signal errors, using built‑in or custom exception types.
- **Custom exceptions** help you convey domain‑specific error information.
- **Exception filters** (`when`) give you fine‑grained control over which catch blocks execute.
- **Best practices** guide you to use exceptions appropriately, preserve stack traces, and avoid common pitfalls.

With exception handling, you can build applications that fail gracefully, provide useful feedback, and maintain data integrity.

In the next chapter, **LINQ (Language Integrated Query) Deep Dive**, we'll explore LINQ in depth – from query syntax and method syntax to deferred execution and advanced operators. You'll learn how to query collections, XML, and databases with a consistent, readable syntax.

**Exercises:**

1. Write a method that divides two numbers and handles `DivideByZeroException`. Also handle `OverflowException` if the numbers are large and you use `checked`.
2. Create a custom exception `InvalidAgeException` that includes the invalid age value. Use it in a `Person` class when setting age.
3. Write a program that reads a list of integers from a file, but handles cases where the file doesn't exist, lines contain invalid data, or numbers are out of range. Use exception filters to differentiate between file‑not‑found and other I/O errors.
4. Implement a retry loop: try an operation up to 3 times, with a delay between attempts, catching specific exceptions (e.g., `TimeoutException`) and only retrying on those.
5. Refactor a piece of code that uses `catch (Exception)` and replace it with more specific catches. Add logging in each catch.

Now, get ready to master LINQ in Chapter 15!